In [ ]:
!pip install sovai[full]

In [1]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

In [2]:
tickers_meta = pd.read_parquet("data/tickers.parq")

In [16]:
df_movies = sov.data("movies/boxoffice")

In [17]:
# Reset index to convert 'ticker' and 'date' from MultiIndex to columns

# Ensure 'date' is in datetime format
df_movies['date'] = pd.to_datetime(df_movies['date'])


# Keep only rows where 'date' is within [start_date, max_date]
df_movies = df_movies[df_movies['date'] >= (df_movies["date"].max() - pd.Timedelta(weeks=1))]



In [18]:
df_movies = df_movies[df_movies["ticker"]!="Private"]

In [22]:
df_movies

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,days_in_release,parent company,distributor address,distributorwebsite,release_date
34416,CMCSA,2024-10-16,Speak No Evil,Universal,148175,-0.300,-0.460,1602.000,92.000,35825505,34,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-12
34417,CMCSA,2024-10-16,The Wild Robot,Universal,1040625,-0.360,-0.190,3854.000,270.000,90468510,20,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
34418,CMCSA,2024-10-17,Speak No Evil,Universal,86685,-0.410,-0.580,1602.000,54.000,35912190,35,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-12
34419,CMCSA,2024-10-17,The Wild Robot,Universal,1148605,0.100,-0.210,3854.000,298.000,91617115,21,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
34420,CMCSA,2024-10-18,The Wild Robot,Universal,2810000,1.450,-0.270,3829.000,734.000,94427115,22,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230433,WBD,2024-10-21,Joker: Folie à Deux,Warner Bros.,195907,-0.650,-0.790,2857.000,69.000,56605655,18,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-10-03
230434,WBD,2024-10-22,Beetlejuice Beetlejuice,Warner Bros.,495497,0.360,-0.340,3251.000,152.000,284833442,47,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-09-05
230435,WBD,2024-10-22,Joker: Folie à Deux,Warner Bros.,304139,0.550,-0.640,2857.000,106.000,56909794,19,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-10-03
230436,WBD,2024-10-23,Beetlejuice Beetlejuice,Warner Bros.,367156,-0.260,-0.320,3251.000,113.000,285200598,48,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-09-05


In [27]:
import pandas as pd

# Step 1: Filter out entries where the ticker is "Private"
df_movies = df_movies[df_movies["ticker"] != "Private"]

# Step 2: Ensure 'date' and 'release_date' are datetime objects
df_movies['date'] = pd.to_datetime(df_movies['date'])
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

# Optional: Handle missing data if necessary
# df_movies = df_movies.dropna(subset=['title', 'date', 'gross', 'theaters', 'days_in_release'])

# Step 3: Create a 'week' column representing the start of each week (Monday)
# Calculate the number of days to subtract to get to Monday
df_movies['week'] = df_movies['date'] - pd.to_timedelta(df_movies['date'].dt.weekday, unit='d')

# Verify the 'week' column
print("Sample 'week' column:")
print(df_movies[['date', 'week']].head())

# Step 4: Verify if each 'title' is associated with a single 'ticker'
title_ticker_counts = df_movies.groupby('title')['ticker'].nunique()
duplicate_titles = title_ticker_counts[title_ticker_counts > 1]

if not duplicate_titles.empty:
    print("\nWarning: The following titles are associated with multiple tickers:")
    print(duplicate_titles)
    # Group by both 'title' and 'ticker' to handle duplicates
    group_by_columns = ['title', 'ticker', 'week']
else:
    group_by_columns = ['title', 'week']

# Step 5: Group by 'title' (and 'ticker' if necessary) and 'week', then aggregate key statistics
weekly_stats = (
    df_movies
    .groupby(group_by_columns)
    .agg(
        total_gross=('gross', 'sum'),
        avg_gross=('gross', 'mean'),
        total_theaters=('theaters', 'sum'),
        avg_theaters=('theaters', 'mean'),
        total_days_in_release=('days_in_release', 'sum'),
        avg_percent_yd=('percent_yd', 'mean'),
        avg_percent_lw=('percent_lw', 'mean')
    )
    .reset_index()
    .sort_values(['title'] + (['ticker'] if 'ticker' in group_by_columns else []) + ['week'])
)

# Step 6: Format numbers for better readability
weekly_stats['total_gross'] = weekly_stats['total_gross'].map('${:,.2f}'.format)
weekly_stats['avg_gross'] = weekly_stats['avg_gross'].map('${:,.2f}'.format)
weekly_stats['total_theaters'] = weekly_stats['total_theaters'].astype(int)
weekly_stats['avg_theaters'] = weekly_stats['avg_theaters'].round(1)
weekly_stats['total_days_in_release'] = weekly_stats['total_days_in_release'].astype(int)
weekly_stats['avg_percent_yd'] = (weekly_stats['avg_percent_yd'] * 100).round(2).astype(str) + '%'
weekly_stats['avg_percent_lw'] = (weekly_stats['avg_percent_lw'] * 100).round(2).astype(str) + '%'

# Optional: Reorder columns for better presentation
columns_order = ['title']
if 'ticker' in group_by_columns:
    columns_order.append('ticker')
columns_order += ['week', 'total_gross', 'avg_gross', 'total_theaters',
                 'avg_theaters', 'total_days_in_release', 'avg_percent_yd', 'avg_percent_lw']

weekly_stats = weekly_stats[columns_order]



Sample 'week' column:
            date       week
34416 2024-10-16 2024-10-14
34417 2024-10-16 2024-10-14
34418 2024-10-17 2024-10-14
34419 2024-10-17 2024-10-14
34420 2024-10-18 2024-10-14


In [29]:
group_by_columns

['title', 'week']

In [28]:
weekly_stats

,title,week,total_gross,avg_gross,total_theaters,avg_theaters,total_days_in_release,avg_percent_yd,avg_percent_lw
0,Alien: Romulus,2024-10-14,"$32,524.00","$6,504.80",285,57.000,320,-5.8%,-71.8%
1,Alien: Romulus,2024-10-21,"$6,812.00","$2,270.67",105,35.000,204,-18.67%,-76.67%
2,Beetlejuice Beetlejuice,2024-10-14,"$6,045,728.00","$1,209,145.60",16569,3313.800,215,30.4%,-30.0%
3,Beetlejuice Beetlejuice,2024-10-21,"$1,226,347.00","$408,782.33",9753,3251.000,141,-21.0%,-45.67%
4,It Ends With Us,2024-10-14,"$59,353.00","$11,870.60",1407,281.400,355,130.4%,89.8%
5,It Ends With Us,2024-10-21,"$31,453.00","$10,484.33",1299,433.000,225,-14.33%,264.0%
6,Joker: Folie à Deux,2024-10-14,"$3,117,238.00","$623,447.60",16775,3355.000,75,1.4%,-65.8%
7,Joker: Folie à Deux,2024-10-21,"$704,310.00","$234,770.00",8571,2857.000,57,-14.33%,-68.0%
8,Megalopolis,2024-10-14,"$114,558.00","$28,639.50",604,151.000,88,-8.5%,-72.75%
9,Megalopolis,2024-10-21,"$39,936.00","$13,312.00",225,75.000,78,-10.0%,-68.33%


In [20]:
df_movies["ticker"].value_counts()

ticker
SONY     24
CMCSA    16
LGF.A    16
WBD      16
PARA     15
DIS       8
Name: count, dtype: int64

In [7]:
df_movies

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,days_in_release,parent company,distributor address,distributorwebsite,release_date
34416,CMCSA,2024-10-16,Speak No Evil,Universal,148175,-0.300,-0.460,1602.000,92.000,35825505,34,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-12
34417,CMCSA,2024-10-16,The Wild Robot,Universal,1040625,-0.360,-0.190,3854.000,270.000,90468510,20,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
34418,CMCSA,2024-10-17,Speak No Evil,Universal,86685,-0.410,-0.580,1602.000,54.000,35912190,35,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-12
34419,CMCSA,2024-10-17,The Wild Robot,Universal,1148605,0.100,-0.210,3854.000,298.000,91617115,21,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
34420,CMCSA,2024-10-18,The Wild Robot,Universal,2810000,1.450,-0.270,3829.000,734.000,94427115,22,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230433,WBD,2024-10-21,Joker: Folie à Deux,Warner Bros.,195907,-0.650,-0.790,2857.000,69.000,56605655,18,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-10-03
230434,WBD,2024-10-22,Beetlejuice Beetlejuice,Warner Bros.,495497,0.360,-0.340,3251.000,152.000,284833442,47,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-09-05
230435,WBD,2024-10-22,Joker: Folie à Deux,Warner Bros.,304139,0.550,-0.640,2857.000,106.000,56909794,19,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-10-03
230436,WBD,2024-10-23,Beetlejuice Beetlejuice,Warner Bros.,367156,-0.260,-0.320,3251.000,113.000,285200598,48,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-09-05


In [5]:
df_movies.sort_values("date")

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,days_in_release,parent company,distributor address,distributorwebsite,release_date
192093,WBD,1997-07-01,Head Above Water,Fine Line,1588,-0.130,0.000,10.000,159.000,20425,7,Warner Bros. Discovery,"Burbank, California, United States",None,1997-06-24
8370,CMCSA,1997-07-01,The Lost World: Jurassic …,Universal,681480,-0.100,-0.350,2607.000,261.000,214658249,40,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,1997-05-22
166257,SONY,1997-07-01,My Best Friend's Wedding,Sony Pictures,2320386,0.010,-0.310,2134.000,1087.000,53842818,12,Sony Corporation,"10202 West Washington Boulevard, Culver City, ...",https://www.sonypictures.com/,1997-06-19
166256,SONY,1997-07-01,Men in Black,Sony Pictures,4815427,0.000,0.000,0.000,0.000,4815427,0,Sony Corporation,"10202 West Washington Boulevard, Culver City, ...",https://www.sonypictures.com/,1997-07-01
166255,SONY,1997-07-01,The Fifth Element,Sony Pictures,42025,-0.320,-0.710,662.000,63.000,60756457,54,Sony Corporation,"10202 West Washington Boulevard, Culver City, ...",https://www.sonypictures.com/,1997-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34430,CMCSA,2024-10-23,The Wild Robot,Universal,696195,-0.350,-0.330,3829.000,182.000,104203520,27,Comcast Corporation,"10 Universal City Plaza, Universal City, Calif...",https://www.universalpictures.com///,2024-09-26
114846,PARA,2024-10-23,Transformers One,Paramount Pi…,129873,-0.320,-0.530,2169.000,60.000,57130164,34,Paramount Global,"1515 Broadway, New York City, New York, 10036,...",http://www.paramount.com/,2024-09-19
163511,Private,2024-10-23,Goodrich,Ketchup Ente…,105391,-0.350,0.000,1055.000,100.000,986243,6,Ketchup Entertainment,"13949 Ventura Boulevard #230,Sherman Oaks,Cali...",https://www.ketchupentertainment.com/,2024-10-17
230437,WBD,2024-10-23,Joker: Folie à Deux,Warner Bros.,204264,-0.330,-0.610,2857.000,71.000,57114058,20,Warner Bros. Entertainment Inc.,"4000 Warner Blvd., Burbank, California, US",https://www.warnerbros.com/,2024-10-03


In [93]:
df_wiki = sov.data("wikipedia/views", full_history=True)

In [ ]:

# ================================
# Step 1: Data Preparation
# ================================

# Reset index to convert 'ticker' and 'date' from MultiIndex to columns
df_wiki = df_wiki.reset_index()

# Ensure 'date' is in datetime format
df_wiki['date'] = pd.to_datetime(df_wiki['date'])


# Keep only rows where 'date' is within [start_date, max_date]
df_wiki = df_wiki[df_wiki['date'] >= (df_wiki["date"].max() - pd.Timedelta(weeks=12))]



# Sort the DataFrame by 'ticker' and 'date' to maintain chronological order
df_daily = df_wiki.sort_values(['ticker', 'date']).copy()

# Calculate 'previous_day_search_pressure' by shifting 'search_pressure' by one day within each 'ticker'
df_daily['prev_day'] = df_daily.groupby('ticker')['search_pressure'].shift(1)

# Calculate 'daily_change' as the difference between current and previous search pressures
df_daily['day_chg'] = df_daily['search_pressure'] - df_daily['prev_day']

# Calculate 'average_search_quarter' as the average 'search_pressure' over the last 12 weeks per 'ticker'
df_daily['avg_day'] = df_daily.groupby('ticker')['search_pressure'].transform('mean')

# Calculate 'long_term_change' as the difference between current 'search_pressure' and 'average_search_quarter'
df_daily['ma_day'] = df_daily['search_pressure'] - df_daily['avg_day']

# Drop rows with NaN values (e.g., first day per ticker where 'previous_search_pressure' is NaN)
# Filter to include only rows where 'date' == 'max_date_per_ticker' per ticker
df_daily = df_daily[df_daily['date'] == df_wiki["date"].max() ]

df_daily = df_daily.rename(columns={"search_pressure":"day_search"})

# Select relevant columns
df_daily = df_daily[['ticker', 'date', 'day_search',  
                     'day_chg', 'ma_day']]



import pandas as pd

df_wiki = df_wiki.set_index(["ticker","date"])

# Step 1: Find the global maximum date
max_date = df_wiki.index.get_level_values('date').max()

# Step 2: Get the weekday abbreviation (e.g., 'TUE' for Tuesday)
weekday_abbr = max_date.strftime('%a').upper()[:3]

# Define the resampling frequency to end on the max_date's weekday
resample_freq = f'W-{weekday_abbr}'

df_wiki = (
    df_wiki
    .groupby('ticker')
    .resample(resample_freq, level='date')['search_pressure']
    .mean()
    .reset_index()
)


# Step 1: Sort the DataFrame by 'ticker' and 'date'
df_wiki = df_wiki.sort_values(['ticker', 'date'])


# Step 3: Create 'search_past' by shifting 'search_pressure' by one week within each 'ticker'
df_wiki['search_past'] = df_wiki.groupby('ticker')['search_pressure'].shift(1)

# Step 4: Calculate 'search_change' as the difference between current and past search pressures
df_wiki['search_change'] = df_wiki['search_pressure'] - df_wiki['search_past']

# Optional: Handle NaN values if desired
# df_wiki['search_past'] = df_wiki['search_past'].fillna(0)
# df_wiki['search_change'] = df_wiki['search_change'].fillna(0)

import pandas as pd

# Assuming df_wiki is already processed up to the previous steps

# Step 1: Calculate 'search_quarter' as the average 'search_pressure' over the 12 weeks per 'ticker'
df_wiki['search_quarter'] = df_wiki.groupby('ticker')['search_pressure'].transform('mean')

# Step 2: Calculate 'lt_change' as the difference between 'search_pressure' and 'search_quarter'
df_wiki['lt_change'] = df_wiki['search_pressure'] - df_wiki['search_quarter']

# Optional: If you prefer to round the results for better readability
# df_wiki['search_quarter'] = df_wiki['search_quarter'].round(3)
# df_wiki['lt_change'] = df_wiki['lt_change'].round(3)

# Display the updated DataFrame
print(df_wiki.head())


df_wiki = df_wiki.dropna()



# Filter out rows where 'date' is the max date for each 'ticker'
df_wiki = df_wiki[df_wiki['date'] == df_wiki["date"].max()]


df_wiki.sort_values("search_change")

df_wiki = df_wiki.drop(columns=["search_quarter"])

df_wiki = df_wiki.rename(columns={"search_pressure":"search", "search_past":"last","search_change":"change","lt_change":"ltavg"})

df_wiki = pd.merge(df_wiki.set_index(["ticker","date"]), df_daily.set_index(["ticker","date"]),left_index=True, right_index=True)

df_wiki.head()

import pandas as pd

# Rename columns
df_wiki = df_wiki.rename(columns={
    'day_chg': 'day change',
    'change': 'week change',
    'ma_day': 'short',
    'ltavg': 'long'
})

# Reorder columns with day metrics first, then week metrics
new_order = ['search', 'last', 'week change', 'long', 'day change', 'short']

df_wiki = df_wiki[new_order]

In [44]:
df_org = sov.data("wikipedia/views", full_history=True)

In [106]:
import pandas as pd
import numpy as np

def create_pressure_history_columns(df_org, df_wiki, lookback=60):
    """
    Creates pressure history columns from df_org and merges them with df_wiki.

    Parameters:
    - df_org (pd.DataFrame): Original DataFrame with a MultiIndex including 'date'.
    - df_wiki (pd.DataFrame): DataFrame to merge the pressure columns into.
    - lookback (int): Number of days to look back for pressure data.

    Returns:
    - pd.DataFrame: Merged DataFrame with pressure history columns.
    """
    # 1. Get the maximum date from index level 'date'
    max_date = df_org.index.get_level_values('date').max()
    
    # 2. Calculate the cutoff date
    cutoff_date = max_date - pd.Timedelta(days=lookback)
    
    # 3. Filter df_org to only include data after cutoff date
    df_filtered = df_org[df_org.index.get_level_values('date') >= cutoff_date]
    
    # 4. Create pivot table with filtered data
    df_pivot = df_filtered.reset_index().pivot(
        index='ticker', 
        columns='date', 
        values='search_pressure'
    )
    
    # 5. Sort the columns by date ascendingly to ensure pressure_0 is the earliest
    df_pivot = df_pivot.sort_index(axis=1)
    
    # 6. Create pressure column names in ascending order
    num_cols = len(df_pivot.columns)
    all_pressure_cols = [f'pressure_{i}' for i in range(num_cols)]
    df_pivot.columns = all_pressure_cols
    
    # 7. Select columns: first, every 4th, and last
    pressure_cols_to_keep = [all_pressure_cols[0]]  # First column (pressure_0)
    if len(all_pressure_cols) > 2:  # If we have middle columns
        pressure_cols_to_keep.extend(all_pressure_cols[1:-1:4])  # Every 4th column
    pressure_cols_to_keep.append(all_pressure_cols[-1])  # Last column (pressure_{num_cols-1})
    
    # 8. Keep only selected columns
    df_pivot = df_pivot[pressure_cols_to_keep]
    
    # 9. Reset index to make ticker a column
    df_pivot = df_pivot.reset_index()
    
    # 10. Merge with original df_wiki
    df_wiki_expanded = df_wiki.merge(df_pivot, on='ticker', how='left')
    
    return df_wiki_expanded

# Example Usage:

# Assuming you have df_org and df_wiki already defined
# df_org should have a MultiIndex with 'date' and 'ticker'
# df_wiki is the DataFrame you want to expand with pressure columns

# Apply the function
df_wiki_expanded = create_pressure_history_columns(df_org, df_wiki)

# To check the columns we kept and their order
pressure_columns = [col for col in df_wiki_expanded.columns if col.startswith('pressure_')]
print("Pressure columns kept and their order:", pressure_columns)


Pressure columns kept and their order: ['pressure_0', 'pressure_1', 'pressure_5', 'pressure_9', 'pressure_13', 'pressure_17', 'pressure_21', 'pressure_25', 'pressure_29', 'pressure_33', 'pressure_37', 'pressure_41', 'pressure_45', 'pressure_49', 'pressure_53', 'pressure_57', 'pressure_60']


In [107]:
df_wiki_expanded[["search","last","week change","long","day change","short"]] = df_wiki_expanded[["search","last","week change","long","day change","short"]]*100

In [108]:
import pandas as pd


columns_to_process = ['search', 'week change', 'long', 'day change', 'short']

# Initialize an empty list to store DataFrames
top_bottom_dfs = []

# Iterate over each column
for col in columns_to_process:
    # Ensure the column exists in the DataFrame
    if col not in df_wiki_expanded.columns:
        print(f"Column '{col}' does not exist in the DataFrame.")
        continue

    # Sort ascending to get bottom 15
    bottom_15 = df_wiki_expanded.sort_values(by=col, ascending=True).head(15).copy()
    
    # Sort descending to get top 15
    top_15 = df_wiki_expanded.sort_values(by=col, ascending=False).head(15).copy()
    
    # Append to the list
    top_bottom_dfs.extend([top_15, bottom_15])

# Concatenate all DataFrames in the list
combined_df = pd.concat(top_bottom_dfs, ignore_index=True)

# Remove duplicate rows
combined_df_unique = combined_df.drop_duplicates()

# Reset index for cleanliness
combined_df_unique.reset_index(drop=True, inplace=True)


In [109]:
combined_df_unique = combined_df_unique.sample(100)

In [110]:
combined_df_unique = combined_df_unique.drop(columns=["long"])

In [111]:
combined_df_unique = combined_df_unique.rename(columns={"short":"long"})

In [112]:
combined_df_unique

,ticker,search,last,week change,day change,long,pressure_0,pressure_1,pressure_5,pressure_9,pressure_13,pressure_17,pressure_21,pressure_25,pressure_29,pressure_33,pressure_37,pressure_41,pressure_45,pressure_49,pressure_53,pressure_57,pressure_60
54,IL,48.748,71.561,-22.813,-3.990,-31.412,0.663,0.682,0.717,0.750,0.785,0.808,0.798,0.742,0.770,0.693,0.616,0.644,0.736,0.758,0.633,0.492,0.360
103,OJIPY,54.772,54.473,0.298,-3.963,8.821,0.355,0.355,0.342,0.313,0.321,0.272,0.210,0.245,0.365,0.463,0.452,0.405,0.454,0.537,0.580,0.562,0.471
57,HEW,23.970,46.507,-22.537,-0.927,-35.163,0.567,0.547,0.465,0.358,0.358,0.478,0.617,0.714,0.772,0.722,0.629,0.587,0.544,0.502,0.372,0.231,0.171
62,MSG,78.910,70.917,7.992,0.674,45.952,0.201,0.214,0.215,0.210,0.206,0.237,0.229,0.339,0.481,0.543,0.543,0.598,0.672,0.692,0.746,0.790,0.814
110,KWR,50.528,29.071,21.457,1.802,42.515,0.127,0.113,0.074,0.085,0.107,0.111,0.099,0.125,0.094,0.092,0.154,0.205,0.189,0.262,0.379,0.512,0.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,RENT,76.906,70.448,6.458,-2.184,35.473,0.131,0.124,0.093,0.139,0.200,0.275,0.350,0.398,0.546,0.652,0.676,0.644,0.648,0.688,0.757,0.777,0.739
95,MKFG,45.797,54.299,-8.501,-5.007,-23.255,0.647,0.630,0.541,0.504,0.562,0.545,0.582,0.717,0.796,0.807,0.734,0.650,0.585,0.548,0.527,0.497,0.332
104,LUNR,42.768,52.528,-9.760,-3.949,-20.241,0.562,0.605,0.710,0.771,0.758,0.684,0.612,0.586,0.718,0.789,0.764,0.656,0.583,0.534,0.497,0.444,0.322
101,GIC,54.063,66.097,-12.034,-4.175,-15.375,0.419,0.433,0.455,0.497,0.571,0.640,0.700,0.771,0.799,0.776,0.699,0.613,0.611,0.667,0.669,0.548,0.416


In [115]:
# Initialize Datawrapper
dw = Datawrapper(access_token="your_token")

# Create the chart
chart = dw.create_chart(
    title="Stock Wikipedia Views Analysis",
    chart_type="tables"
)

# Add the data to the chart
dw.add_data(chart['id'], data=combined_df_unique)

# Get pressure column names
pressure_cols = sorted([col for col in combined_df_unique.columns if col.startswith('pressure_')])

# Configure the visualization properties
properties = {
     "visualize": {
        "dark-mode-invert": True,
        "perPage": 20,
        "columns": {
            "ticker": {
                "align": "left",
                "title": "Stock",
                "width": "100"
            },
            "search": {
                "title": "Search",
                "format": "0.000",
                "width": "120"
            },
            "last": {
                "title": "Last",
                "format": "0.000",
                "width": "120"
            },
            "week change": {
                "title": "Change",
                "format": "+0.000",
                "width": 0.27,  # Updated to match working example
                "showAsBar": True,
                "barColorNegative": "#ff4444",
                "fixedWidth": True
            },
            "long": {
                "title": "Long Trend",
                "format": "+0.000",
                "width": "120"
            },
            "day change": {
                "title": "Day Change",
                "format": "+0.000",
                "width": 0.17,  # Updated to match working example
                "fixedWidth": True
            },
            "pressure_0": {
                "type": "number",
                "title": "History",  # Updated to just "History"
                "width": 0.33,  # Updated to match working example
                "format": "0.000",
                "sparkline": {
                    "color": "#18a1cd",
                    "title": "History",
                    "enabled": True,
                    "stroke": 2,
                    "dotMax": True,
                    "dotMin": True,
                    "dotFirst": True,
                    "dotLast": True
                },
                "fixedWidth": True
            }
        },
        "header": {
            "style": {
                "bold": True,
                "fontSize": 0.9,
                "color": "#494949"
            },
            "borderBottom": "2px",
            "borderBottomColor": "#333333"
        },
        "pagination": {
            "enabled": True,
            "position": "bottom",
            "pagesPerScreen": 10
        },
        "striped": True,
        "markdown": True,
        "showHeader": True,
        "compactMode": True,
        "firstRowIsHeader": False,
        "firstColumnIsSticky": False,
        "mergeEmptyCells": False
    }
}

# Configure remaining pressure columns exactly like the first one
for col in pressure_cols[1:]:
    properties["visualize"]["columns"][col] = {
        "type": "number",
        "width": 0.33,  # Updated to match working example
        "format": "0.000",
        "sparkline": {
            "color": "#18a1cd",
            "title": "pressure_history",
            "enabled": True
        },
        "fixedWidth": True
    }

# Set column order
properties["visualize"]["column-order"] = [
    "ticker",
    "search",
    "last",
    "weel change",
    "long",
    "day change",
] + pressure_cols

# Add other visualization settings
properties["describe"] = {
    "intro": "Analysis of Wikipedia page views for stocks with historical pressure trends over 60 days.",
    "byline": "",
    "source-name": "Wikipedia Views Data",
    "source-url": "",
    "hide-title": False
}

properties["publish"] = {
    "embed-width": 682,
    "embed-height": 1086,
    "blocks": {
        "logo": {"enabled": False},
        "embed": False,
        "download-pdf": False,
        "download-svg": False,
        "get-the-data": True,
        "download-image": False
    },
    "autoDarkMode": False,
    "chart-height": 988,
    "force-attribution": False
}

# Update the chart with the properties
dw.update_chart(
    chart['id'],
    metadata=properties
)

# Publish the chart
dw.publish_chart(chart['id'])

# Get the published URL
published_url = dw.get_chart_display_urls(chart['id'])
print("Published Chart URL:", published_url)

Published Chart URL: [{'id': 'standalone', 'url': 'https://www.datawrapper.de/_/Dehx9/', 'name': 'For sharing'}]


In [114]:
dw.publish_chart("01Wlv")

{'data': {'publicId': '01Wlv',
  'language': 'en-US',
  'theme': 'datawrapper',
  'id': '01Wlv',
  'type': 'tables',
  'title': 'Stock Wikipedia Views Analysis',
  'lastEditStep': 5,
  'publishedAt': '2024-10-24T12:04:00.930Z',
  'publicUrl': 'https://datawrapper.dwcdn.net/01Wlv/3/',
  'publicVersion': 3,
  'deleted': False,
  'deletedAt': None,
  'forkable': False,
  'isFork': False,
  'metadata': {'data': {'changes': [],
    'transpose': False,
    'vertical-header': True,
    'horizontal-header': True,
    'upload-method': 'copy'},
   'describe': {'source-name': 'Wikipedia Views Data',
    'source-url': '',
    'intro': 'Analysis of Wikipedia page views for stocks with historical pressure trends over 60 days.',
    'byline': '',
    'aria-description': '',
    'number-format': '-',
    'number-divisor': 0,
    'number-append': '',
    'number-prepend': '',
    'hide-title': False},
   'visualize': {'dark-mode-invert': True,
    'highlighted-series': [],
    'highlighted-values': [],

In [22]:
df_shorted.head()

,ticker,date,over_shorted,change,short_interest,actual,expected,dtc,market_cap
0,WRAP,2024-10-11,53.200,2.400,5237964.000,81.300,28.100,0.140,71.600
1,NLOP,2024-10-11,51.900,8.300,2371407.000,84.700,32.800,0.278,447.000
2,CPSS,2024-10-11,48.800,2.400,1135987.000,68.100,19.300,0.183,202.000
3,CNQ,2024-10-11,48.500,2.600,192390720.000,77.900,29.400,0.111,79680.102
4,GV,2024-10-11,48.500,-2.300,461139.000,81.000,32.600,0.488,7.200


In [3]:
### Here make space for some utility functions.
import os
from dotenv import load_dotenv
from github import Github
import plotly.graph_objects as go

def upload_file_to_github(plot_html, file_path_in_repo, commit_message="Add/update plot",
                         env_path='/Users/dereksnow/Sovai/GitHub/SovAI/.env', repo_name="sovai-research/sovai-research.github.io", git_token_env_var='GIT_TOKEN'):
    """
    Uploads or updates a file in the specified GitHub repository.

    Parameters:
    - plot_html (str): The content of the file to upload (e.g., HTML string).
    - file_path_in_repo (str): The path in the repository where the file will be uploaded (e.g., 'plots/risks/turing_risk_plot.html').
    - commit_message (str): The commit message. Defaults to "Add/update plot".
    - env_path (str, optional): The path to the .env file. If None, load_dotenv() will search automatically.
    - repo_name (str, optional): The full name of the repository (e.g., "owner/repo"). If None, read from environment variable 'GIT_REPO_NAME'.
    - git_token_env_var (str, optional): The name of the environment variable that holds the GitHub token. Defaults to 'GIT_TOKEN'.

    Returns:
    - None

    Raises:
    - ValueError: If the GitHub token or repository name is not provided.
    - Exception: For other unexpected errors during the upload/update process.
    """

    # Load environment variables
    if env_path:
        load_dotenv(dotenv_path=env_path)
    else:
        load_dotenv()  # Automatically load from .env in current or parent directories

    # Retrieve GitHub token from environment variables
    GIT_TOKEN = os.getenv(git_token_env_var)
    if not GIT_TOKEN:
        raise ValueError(f"{git_token_env_var} not found in environment variables. Please set it in the .env file or environment.")

    # Initialize GitHub client
    g = Github(GIT_TOKEN)

    # Retrieve repository name
    if not repo_name:
        repo_name = os.getenv('GIT_REPO_NAME')
        if not repo_name:
            raise ValueError("Repository name not specified and 'GIT_REPO_NAME' not found in environment variables.")

    # Initialize repository
    try:
        repo = g.get_repo(repo_name)
        print(f"Successfully connected to repository: {repo.full_name}")
    except Exception as e:
        print(f"Error accessing repository '{repo_name}': {e}")
        raise e

    # Upload or update the file
    try:
        # Attempt to get the existing file
        contents = repo.get_contents(file_path_in_repo)
        # If the file exists, update it
        repo.update_file(contents.path, commit_message, plot_html, contents.sha)
        print(f"File '{file_path_in_repo}' updated successfully.")
    except Exception as e:
        if "404" in str(e):
            # If the file does not exist, create it
            repo.create_file(file_path_in_repo, commit_message, plot_html)
            print(f"File '{file_path_in_repo}' created successfully.")
        else:
            print(f"An error occurred: {e}")
            raise e


In [4]:
df_congress = sov.data("congress")

In [21]:
import pandas as pd
from datetime import timedelta
import requests
import numpy as np

# Convert 'date' and 'transaction_date' to datetime
df_congress['date'] = pd.to_datetime(df_congress['date'])
df_congress['transaction_date'] = pd.to_datetime(df_congress['transaction_date'])

# Find the latest date
latest_date = df_congress['date'].max()
print(f"Latest Date: {latest_date.date()}")

# Calculate start date (7 days before latest_date)
start_date = latest_date - timedelta(days=7)
print(f"Start Date: {start_date.date()}")

# Filter for the last 7 days
df_last_7_days = df_congress[df_congress['date'] >= start_date].copy()
print(f"Number of Transactions in Last 7 Days: {df_last_7_days.shape[0]}")

# Get unique tickers in the last 7 days
unique_tickers = df_last_7_days['ticker'].unique()
print(f"Unique Tickers in Last 7 Days: {unique_tickers}")

# Define base URL for raw GitHub content
base_url = 'https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/'

# Define default logo URL
default_logo_url = 'https://avatars.githubusercontent.com/u/114351196?s=48&v=4'  # Replace with actual default if different

# Function to assign logo URL with error handling
def get_logo_url(ticker):
    url = f"{base_url}{ticker}.png"
    try:
        response = requests.head(url, timeout=5)
        if response.status_code == 200:
            return url
        else:
            return default_logo_url
    except requests.RequestException:
        return default_logo_url

# Create a dictionary mapping tickers to their logo URLs
ticker_logo_mapping = {ticker: get_logo_url(ticker) for ticker in unique_tickers}

# Display the mapping
for ticker, logo_url in ticker_logo_mapping.items():
    print(f"Ticker: {ticker}, Logo URL: {logo_url}")

# Assign logo URLs where 'date' >= start_date, else set to None
df_congress['logo'] = np.where(
    df_congress['date'] >= start_date,
    df_congress['ticker'].map(ticker_logo_mapping),
    None  # Or assign a default value like default_logo_url
)

# Alternatively, using pd.Series.where
# ticker_logo_series = pd.Series(ticker_logo_mapping)
# df_congress['logo'] = df_congress['ticker'].map(ticker_logo_series).where(df_congress['date'] >= start_date)

# Display the updated DataFrame
print(df_congress[['ticker', 'date', 'logo']])

# Filter congress dataframe based on date
df_congress = df_congress[df_congress['date'] >= start_date]

from datawrapper import Datawrapper
import pandas as pd

# Function to get price history for a ticker and convert to columns
def get_price_history_columns(ticker):
    try:
        # Get price data using your SDK
        df_price = sov.data("market/closeadj", tickers=[ticker])
        # Get last 90 days of data
        latest_prices = df_price.tail(90)
        # Convert dates to column names and values to a row
        return latest_prices['closeadj'].values
    except:
        return None

# Create a list to store the data
all_data = []

# Process each ticker
for idx, row in df_congress.iterrows():
    prices = get_price_history_columns(row['ticker'])
    if prices is not None:
        data_dict = {
            'ticker': row['ticker'],
            'logo': row['logo'],
            'transaction_date': row['transaction_date'],
            'representative': row['representative'],
            'bio_guide_url': row['bio_guide_url'],
            'transaction': row['transaction'],
            'amount': row['amount'],
            'house': row['house'],
            'transaction': row['transaction'],
            'party': row['party'],
            'days_to_report': row['days_to_report'],
        }
        # Add price columns
        for i, price in enumerate(prices):
            data_dict[f'price_{i}'] = price
        all_data.append(data_dict)

# Create new dataframe with price columns
df_with_prices_org = pd.DataFrame(all_data)


Latest Date: 2024-10-22
Start Date: 2024-10-15
Number of Transactions in Last 7 Days: 52
Unique Tickers in Last 7 Days: ['V' 'BX' 'BRK.B' 'TSLA' 'INTC' 'CAT' 'MSFT' 'HD' 'NSRGY' 'ASML' 'EFC'
 'IWM' 'SHY' 'MDY' 'IEF' 'IVV' 'FI' 'JMBS' 'EFA' 'VEA' 'FTGC' 'TLT' 'IEI'
 'TLH' 'WWSYX' 'PALL' 'TPYP' 'SCHW' 'CMBS' 'MBB' 'VNQI' 'TBLL' 'GIGB'
 'SCHP' 'EMLP' 'PPLT' 'AGZ' 'LSXMK' 'J' 'AMTM' 'CMG' 'NKE' 'ITW' 'ADP'
 'LSXMA' 'SIRI']
Ticker: V, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/V.png
Ticker: BX, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/BX.png
Ticker: BRK.B, Logo URL: https://avatars.githubusercontent.com/u/114351196?s=48&v=4
Ticker: TSLA, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/TSLA.png
Ticker: INTC, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/INTC.png
Ticker: CAT, Logo URL: https://raw.githubusercontent.com/nvstly/icons/main/ticker_icons/CAT.png
Ticker: MSFT, L

In [47]:
df_with_prices = df_with_prices_org.copy()


# Format the data to include logos and links

# df_with_prices['ticker'] = df_with_prices.apply(
#     lambda row: f"{row['ticker']}<tooltip>{row['logo']}</tooltip>", 
#     axis=1
# )

df_with_prices['logo'] = df_with_prices['logo'].apply(lambda x: f'![logo]({x})')
df_with_prices['representative'] = df_with_prices.apply(
    lambda x: f'[{x["representative"]}]({x["bio_guide_url"]})', axis=1
)


df_with_prices['house_party'] = df_with_prices.apply(
    lambda x: f"{'Sen' if x['house']=='Senate' else 'Rep'} ({x['party']})", 
    axis=1
)


df_with_prices = df_with_prices.drop(columns=["bio_guide_url","party","house","days_to_report","logo"])

# # Format ticker with logo and Yahoo link
df_with_prices['ticker'] = df_with_prices.apply(
    lambda x: f"[{x['ticker']}](https://finance.yahoo.com/quote/{x['ticker']})", 
    axis=1
)

# # # Format ticker with logo and Yahoo link
# df_with_prices['transaction'] = df_with_prices.apply(
#     lambda x: f"[{x['transaction']}: {x['logo']}", 
#     axis=1
# )



# Get the list of column names
columns = df_with_prices.columns

# Filter for price_ columns
price_columns = [col for col in columns if col.startswith('price_')]

# Always keep the first and last price_ columns, and select every 4th column in between
selected_price_columns = [price_columns[0]] + price_columns[1:-1:4] + [price_columns[-1]]

# Define the final columns to keep, including the non-price columns
non_price_columns = [col for col in columns if not col.startswith('price_')]
final_columns = non_price_columns + selected_price_columns

# Create a new dataframe with the selected columns
df_with_prices = df_with_prices[final_columns]


df_with_prices = df_with_prices[[col for col in df_with_prices.columns if col not in ['house_party', 'transaction', 'amount',]] + ['house_party','transaction', 'amount']]

df_with_prices = df_with_prices.rename(columns={"transaction":"type", "transaction_date":"transaction","house_party":"house"})

# Display the result
df_with_prices.head()


,ticker,transaction,representative,price_0,price_1,price_5,price_9,price_13,price_17,price_21,price_25,price_29,price_33,price_37,price_41,price_45,price_49,price_53,price_57,price_61,price_65,price_69,price_73,price_77,price_81,price_85,price_89,house,type,amount
0,[V](https://finance.yahoo.com/quote/V),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,270.119,270.628,275.748,261.946,269.820,262.025,272.155,264.261,261.077,266.047,259.311,261.140,268.040,268.210,276.370,279.370,285.370,288.480,272.780,274.950,277.930,277.470,287.520,284.790,Rep (R),Purchase,1001.0
1,[BX](https://finance.yahoo.com/quote/BX),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,121.250,122.160,125.260,123.800,122.720,126.610,134.850,143.620,140.980,133.050,130.650,136.160,132.980,141.960,142.360,137.120,146.500,154.930,157.550,153.130,151.920,149.030,159.710,168.500,Rep (R),Purchase,1001.0
2,[BRK.B](https://finance.yahoo.com/quote/BRK.B),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,405.540,407.320,413.980,406.800,411.170,418.780,445.610,434.010,438.310,428.360,431.540,438.470,448.360,454.490,475.920,459.420,450.870,456.600,454.560,460.260,461.970,454.930,465.690,461.450,Rep (R),Purchase,1001.0
3,[TSLA](https://finance.yahoo.com/quote/TSLA),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,178.010,187.440,182.580,197.880,251.520,241.030,248.500,246.380,232.100,207.670,198.840,201.380,221.100,213.210,214.110,210.730,229.810,227.200,254.270,261.630,250.080,238.770,221.330,217.970,Rep (R),Purchase,1001.0
4,[INTC](https://finance.yahoo.com/quote/INTC),2024-10-21,[Marjorie Taylor Mrs Greene](https://bioguide....,30.251,30.777,30.370,30.767,31.811,33.281,34.235,32.735,30.628,21.340,20.490,19.920,20.990,20.130,22.040,18.890,19.360,20.770,22.810,23.460,22.590,23.220,22.310,22.400,Rep (R),Purchase,1001.0


In [51]:
# Initialize Datawrapper
dw = Datawrapper(access_token="your_token")

# Create the chart
chart = dw.create_chart(
    title="Congressional Stock Trades with Price History",
    chart_type="tables"
)

# Add the data to the chart
dw.add_data(chart['id'], data=df_with_prices)

# Get price column names
price_cols = [col for col in df_with_prices.columns if col.startswith('price_')]

# Configure the visualization properties
properties = {
    "visualize": {
        "dark-mode-invert": True,
        "perPage": 20,
        "columns": {
            "ticker": {
                "align": "left",
                "title": "Stock",
                "tooltip": {
                    "enabled": True,
                    "template": "{{logo}}"
                }
            },
            "transaction": {
                "title": "Date",
                "format": "YYYY-MM-DD"
            },
            "representative": {
                "title": "Representative",
                "width": 0.23,
                "markdown": True,
                "fixedWidth": True
            },
            "price_0": {  
                "type": "number",
                "title": "Price History",
                "width": 0.28,
                "format": "$.2f",
                "visible": True,
                "sparkline": {
                    "color": "#18a1cd",
                    "title": "stock_price",
                    "enabled": True,
                    "stroke": 2,
                    "dotMax": True,
                    "dotMin": True,
                    "dotFirst": True,
                    "dotLast": True
                },
                "fixedWidth": True
            },
            "type": {
                "title": "Type",
                "customColor": True,
                "customColorBy": "type",
                "customColorText": {
                    "Sale": "#ee493a",
                    "Exchange": "#18a1cd",
                    "Purchase": "#09bb9f",
                    "Sale (Partial)": "#ffb239"
                }
            },
            "amount": {
                "title": "Amount",
                "format": "$0,0",
                "align": "right",
                "showAsBar": True,
                "fixedWidth": True
            },
            "house": {
                "title": "Chamber"
            }
        },
        "header": {
            "style": {
                "bold": True,
                "fontSize": 0.9,
                "color": "#494949"
            },
            "borderBottom": "2px",
            "borderBottomColor": "#333333"
        },
        "pagination": {
            "enabled": True,
            "position": "bottom",
            "pagesPerScreen": 10
        },
        "striped": True,
        "markdown": True,
        "showHeader": True,
        "compactMode": True,
        "firstRowIsHeader": False,
        "firstColumnIsSticky": False,
        "mergeEmptyCells": False
    }
}

# Configure remaining price columns
for col in price_cols[1:]:
    properties["visualize"]["columns"][col] = {
        "type": "number",
        "width": 0.28,
        "format": "$.2f",
        "visible": True,
        "sparkline": {
            "color": "#18a1cd",
            "title": "stock_price",
            "enabled": True
        },
        "fixedWidth": True,
        "showOnMobile": False,
        "showOnDesktop": True
    }

# Set column order
properties["visualize"]["column-order"] = [
    "ticker",
    "transaction",
    "representative",
    "price_0"
] + price_cols[1:] + [
    "type",
    "amount",
    "house"
]

# Add other visualization settings
properties["describe"] = {
    "intro": "Congressional stock trades with 90-day historical price performance.",
    "byline": "",
    "source-name": "Congress Trading Data",
    "source-url": "",
    "hide-title": False
}

properties["publish"] = {
    "embed-width": 682,
    "embed-height": 1086,
    "blocks": {
        "logo": {"enabled": False},
        "embed": False,
        "download-pdf": False,
        "download-svg": False,
        "get-the-data": True,
        "download-image": False
    },
    "autoDarkMode": False,
    "chart-height": 988,
    "force-attribution": False
}

# Update the chart with the properties
dw.update_chart(
    chart['id'],
    metadata=properties
)

# Publish the chart
dw.publish_chart(chart['id'])

# Get the published URL
published_url = dw.get_chart_display_urls(chart['id'])
print("Published Chart URL:", published_url)

Published Chart URL: [{'id': 'standalone', 'url': 'https://www.datawrapper.de/_/cv4aP/', 'name': 'For sharing'}]
